In [1]:
import hypernetx as hnx
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sys

## Load tables

In [2]:
dff = pd.read_pickle("biggerTrans.pkl")

In [3]:
dffwos2 = pd.read_pickle("biggerTransNoSars2.pkl")

In [4]:
dffwos = pd.read_pickle("biggerTransNoSars.pkl")

In [5]:
dffwoc = pd.read_pickle("biggerTransNoCov.pkl")

## Build the hypergraph

In [6]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
Hfwos2 = hnx.Hypergraph.from_dataframe(dffwos2, # the whole dataframe, b and p columns
                                                    #columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [7]:
Hfwos2.shape

(7960, 154)

In [8]:
Hfwos2D = Hfwos2.dual()

In [9]:
Hfwos2D.shape

(154, 7960)

In [11]:
graph = Hfwos2D

In [12]:
import ray

In [13]:
ray.init(num_cpus=7)

2020-06-03 15:48:32,528	INFO resource_spec.py:204 -- Starting Ray with 9.57 GiB memory available for workers and up to 4.81 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-03 15:48:32,878	INFO services.py:1168 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '130.20.211.225',
 'raylet_ip_address': '130.20.211.225',
 'redis_address': '130.20.211.225:35003',
 'object_store_address': '/tmp/ray/session_2020-06-03_15-48-32_512785_3120/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-03_15-48-32_512785_3120/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-06-03_15-48-32_512785_3120'}

In [16]:
@ray.remote
def betweenness(s, graph = graph):
    sbt = hnx.s_betweenness_centrality(graph, s=s)
    sbt = pd.Series(sbt)
    sbt.to_pickle("intermediateNoSars2/" + str(s) + "-betweenness.pkl")
    return(sbt.rename(s))

@ray.remote
def closeness(s, graph = graph):
    scl = hnx.s_harmonic_closeness_centrality(graph, s=s)
    scl = pd.Series(scl)
    scl.to_pickle("intermediateNoSars2/" + str(s) + "-closeness.pkl")
    return(scl.rename(s))    

In [17]:
Hfwos2Dsbt2 = ray.get([betweenness.remote(i) for i in range(51,graph.shape[0])])
Hfwos2Dscl2 = ray.get([closeness.remote(i) for i in range(51,graph.shape[0])])

In [23]:
max([i for i in range(len(Hfwos2Dsbt2)) if len(Hfwos2Dsbt2[i]) > 0])

27

In [20]:
Hfwos2Dsbt2[27].name

78

In [22]:
max([i for i in range(len(Hfwos2Dscl2)) if len(Hfwos2Dscl2[i]) > 0])

55

In [25]:
Hfwos2Dscl2[55].name

106

In [26]:
ray.shutdown()